In [1]:
from PIL import Image
import os
import os.path
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [2]:
data_path = './data/train'
# reading(loading) mat file as array
loaded_mat = sio.loadmat(os.path.join(data_path, 'digitStruct.mat'))

In [3]:
#print(loaded_mat)

In [4]:
#print(loaded_mat['digitStruct'])

In [5]:
bb=loaded_mat['digitStruct']
print(bb['name'])

[[array(['1.png'], dtype='<U5') array(['2.png'], dtype='<U5')
  array(['3.png'], dtype='<U5') ... array(['33400.png'], dtype='<U9')
  array(['33401.png'], dtype='<U9') array(['33402.png'], dtype='<U9')]]


In [6]:
filename=bb['name']
bbox=bb['bbox']
print(filename.shape)
print(bbox.shape)

(1, 33402)
(1, 33402)


In [7]:
print(bbox[0,0].shape)
print(str(filename[0,0].astype(str).squeeze()))

(1, 2)
1.png


In [8]:
sliding_window_size = (32,32)
stride = (16,16)

def intersection(box, bb):
    mask1 = (bb[:,1] <= box[0]) & ((bb[:,1]+bb[:,3]) < (box[0])) & \
            (bb[:,2] <= box[1]) & ((bb[:,2]+bb[:,0]) < (box[1]))
    mask2 = (bb[:,1] <= (box[0]+box[2])) & ((bb[:,1]+bb[:,3]) < (box[0]+box[2])) & \
            (bb[:,2] <= (box[1]+box[3])) & ((bb[:,2]+bb[:,0]) < (box[1]+box[3]))
    res = mask1.sum() + mask2.sum()
    return res != 0
        
total = 0
output_path = './data/not_digit'
for i in tqdm(range(filename.shape[1])):
    image = Image.open(os.path.join(data_path,str(filename[0,i].astype(str).squeeze())))
    bbs = bbox[0,i]
    bbnp = []
    for bb in bbs:
        for height,left,top,width,label in bb:
            bbnp.append([height[0,0].astype(int).squeeze(),left[0,0].astype(int).squeeze(),\
                         top[0,0].astype(int).squeeze(),width[0,0].astype(int).squeeze(),\
                         label[0,0].astype(int).squeeze()])
    bbnp = np.array(bbnp)
    #print(bbnp)
    j = 0
    k = 0
    
    W,H = image.size
    
    while (j + sliding_window_size[0]) < W:
        while (k + sliding_window_size[1]) < H:
            box = (j,k,j+sliding_window_size[0],k+sliding_window_size[1])  
            if intersection(box, bbnp) == False:
                region = image.crop(box)
                outfile = os.path.join(output_path,str(total).zfill(8)+'.png')
#                 print(outfile)
                region.save(outfile)
                total+=1
                #plt.figure()
                #plt.imshow(region)
            k = k + stride[1]
        j = j + stride[0]
    #print(str(filename[0,i].astype(str).squeeze()))
print(total)


68736


In [9]:
a = 1
b=str(a).zfill(4)
c = str((a))
os.path.join(b,'ss')

'0001/ss'